In [1]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\Administrator\.cache\huggingface\token
Login successful


In [2]:
#!pip install huggingface_hub
#!pip install datasets
#!pip install transformers
#!pip install evaluate
#!pip install rouge_score

In [12]:
from numpy.lib.shape_base import column_stack
from datasets import load_dataset,Dataset
import pandas as pd
import json
f = open('csl_data.json',encoding="utf-8")
data = json.load(f)
f.close()
df = pd.DataFrame(data)
df['summary'] = df['title']
df.columns = ['title','text','summary']
df = df[['text','summary','title']]
df = Dataset.from_pandas(df)
df = df.train_test_split(test_size=0.2)
df["train"]

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 2800
})

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained("uer/t5-small-chinese-cluecorpussmall")
model = AutoModel.from_pretrained("uer/t5-small-chinese-cluecorpussmall")

Some weights of the model checkpoint at uer/t5-small-chinese-cluecorpussmall were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_df = df.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [17]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [8]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_new_title_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    #fp16=True,
    #push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_df["train"],
    eval_dataset=tokenized_df["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `T5Model.forward` and have been ignored: text, labels, token_type_ids, title, summary. If text, labels, token_type_ids, title, summary are not expected by `T5Model.forward`,  you can safely ignore this message.
c:\Users\Administrator\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1582
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 396
  Number of trainable parameters = 54925824


  0%|          | 0/396 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 4.00 GiB total capacity; 3.33 GiB already allocated; 0 bytes free; 3.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_awesome_new_title_model",
    tokenizer=tokenizer,
)

In [24]:
df['train']['text'][0]

'提出了将基于CCCII的模拟电感用于蔡氏混沌电路,并用Pspice软件对其进行了仿真,实验表明,这种方法是可行的,通过参数的改变,电路展示出了丰富的混沌变化过程。还分析给出了相关元件对电路的影响以及在电路处于混沌状态时,各相关元件的参数变化范围。'

In [26]:
from transformers import pipeline

summarizer = pipeline("summarization", model="skywalker0803r/my_awesome_new_title_model")
text = df['train']['text'][0]
summarizer(text)

[{'summary_text': 'extra0 extra1'}]